# Step 2: Generate data samples

In this section, we will use the [AmplitudeModel](expertsystem.amplitude.model.AmplitudeModel) that we created with the expert system in [the previous step](./1_create_model) to generate a data sample via hit & miss Monte Carlo. We do this with the [tensorwaves.data.generate](tensorwaves.data.generate) module.

First, we load a [AmplitudeModel](expertsystem.amplitude.model.AmplitudeModel) that was created in the previous step:

In [ ]:
from expertsystem import io

model = io.load_amplitude_model("amplitude_model_helicity.yml")

## 2.1 Define kinematics of the particle reaction

An [AmplitudeModel](expertsystem.amplitude.model.AmplitudeModel) defines the kinematics, the particles involved in the reaction, the dynamics used for the model on which to perform the eventual optimization, etc.

In [ ]:
from tensorwaves.physics.helicity_formalism.kinematics import HelicityKinematics

kin = HelicityKinematics.from_model(model)
print("Initial state mass:", kin.reaction_kinematics_info.initial_state_masses)
print("Final state masses:", kin.reaction_kinematics_info.final_state_masses)
print("Involved particles:", model.particles.names)

A [Kinematics](tensorwaves.interfaces.Kinematics) object defines the kinematics of the particle reaction we are studying. From the final state masses listed here, we can see we are dealing with the reaction $J/\psi \to \gamma\pi^0\pi^0$.

## 2.2 Generate phase space sample

[Kinematics](tensorwaves.interfaces.Kinematics) define the constraints of the phase space. As such, we have enough information to generate a **phase-space sample** for this particle reaction. We do this with the [generate_phsp](tensorwaves.data.generate.generate_phsp) function. By default, this function uses [TFPhaseSpaceGenerator](tensorwaves.data.tf_phasespace.TFPhaseSpaceGenerator) as a, well... phase-space generator (using tensorflow in the back-end) and generates random numbers with [TFUniformRealNumberGenerator](tensorwaves.data.tf_phasespace.TFUniformRealNumberGenerator). You can specify this with the arguments of [generate_phsp](tensorwaves.data.generate.generate_phsp) function.

In [ ]:
from tensorwaves.data.generate import generate_phsp

phsp_sample = generate_phsp(300000, kin)
phsp_sample.shape

As you can see, the phase-space sample is a three-dimensional array: **300.000 events** of **four**-momentum tuples for **three** particles.

## 2.3 Generate intensity-based sample

'Data samples' are more complicated than phase space samples in that they represent the intensity profile resulting from a reaction. You therefore need an [IntensityTF](tensorwaves.physics.helicity_formalism.amplitude.IntensityTF) object (or, more generally, a [Function](tensorwaves.interfaces.Function) instance) and a phase space over which to generate that intensity distribution. We call such a data sample an **intensity-based sample**.

An intensity-based sample is generated with the function [generate_data](tensorwaves.data.generate.generate_data). Its usage is similar to [generate_phsp](tensorwaves.data.generate.generate_phsp), but now you have to give an [IntensityTF](tensorwaves.physics.helicity_formalism.amplitude.IntensityTF) in addition to the [Kinematics](tensorwaves.interfaces.Kinematics) object. An [IntensityTF](tensorwaves.physics.helicity_formalism.amplitude.IntensityTF) object can be created with the [IntensityTF](tensorwaves.physics.helicity_formalism.amplitude.IntensityTF) class:

In [ ]:
from tensorwaves.physics.helicity_formalism.amplitude import IntensityBuilder

builder = IntensityBuilder(model.particles, kin, phsp_sample)
intensity = builder.create_intensity(model)

That's it, now we have enough info to create an intensity-based data sample. Notice how the structure is the sample as the phase-space sample we saw before:

In [ ]:
from tensorwaves.data.generate import generate_data

data_sample = generate_data(30000, kin, intensity)
data_sample.shape

## 2.4 Visualize kinematic variables

We now have a phase space sample and an intensity-based sample. Their data structure isn't the most informative though: it's just a collection of four-momentum tuples. However, we can use the [Kinematics](tensorwaves.interfaces.Kinematics) class to convert those four-momentum tuples to a data set of kinematic variables.

Now we can use the [Kinematics.convert](tensorwaves.interfaces.Kinematics.convert) method to convert the phase space and data samples of four-momentum tuples to kinematic variables.

In [ ]:
phsp_set = kin.convert(phsp_sample)
data_set = kin.convert(data_sample)
list(data_set)

The data set is just a [dict](dict) of kinematic variables (keys are the names, values is a list of computed values for each event). The numbers you see here are final state IDs as defined in the [AmplitudeModel](expertsystem.amplitude.model.AmplitudeModel) member of the [AmplitudeModel](expertsystem.amplitude.model.AmplitudeModel):

In [ ]:
for state_id, particle in model.kinematics.final_state.items():
    print(f"ID {state_id}:", particle.name)

````{admonition} Available kinematic variables
---
class: dropdown
---
By default, {mod}`tensorwaves` only generates invariant masses of the {class}`.SubSystem`s that are of relevance to the decay problem. In this case, we only have resonances $f_0 \to \pi^0\pi^0$. If you are interested in more invariant mass combinations, you can do so with the method {meth}`.HelicityKinematics.register_invariant_mass`, e.g.:

```{code-block} python
kin.register_invariant_mass([2, 4])
```
````

The data set is [dict](dict), which allows us to easily convert it to a [pandas.DataFrame](pandas.DataFrame) and plot its content in the form of a histogram:

In [ ]:
import pandas as pd

data_frame = pd.DataFrame(data_set)
phsp_frame = pd.DataFrame(phsp_set)
data_frame

This also means that we can use all kinds of fancy plotting functionality of for instance [matplotlib.pyplot](matplotlib.pyplot) to see what's going on. Here's an example:

In [ ]:
import numpy as np
from matplotlib import cm

intermediate_states = sorted(
    (
        p
        for p in model.particles
        if p not in model.kinematics.final_state.values()
        and p not in model.kinematics.initial_state.values()
    ),
    key=lambda p: p.mass,
)

evenly_spaced_interval = np.linspace(0, 1, len(intermediate_states))
colors = [cm.rainbow(x) for x in evenly_spaced_interval]

In [ ]:
import matplotlib.pyplot as plt

np.sqrt(data_frame["mSq_3_4"]).hist(bins=100, alpha=0.5, density=True)
plt.xlabel("$m$ [GeV]")
for i, p in enumerate(intermediate_states):
    plt.axvline(x=p.mass, linestyle="dotted", label=p.name, color=colors[i])
plt.legend();

## 2.5 Export data sets

[tensorwaves](tensorwaves) currently has no export functionality for data samples. However, as we work with [numpy.ndarray](numpy.ndarray), it's easy to just ['pickle'](https://docs.python.org/3/library/pickle.html) these data samples with [numpy.save](numpy.save):

In [ ]:
import numpy as np

np.save("data_sample", data_sample)
np.save("phsp_sample", phsp_sample)

In the [next step](./3_perform_fit), we will illustrate how to 'perform a fit' with [tensorwaves](tensorwaves) by optimizing the intensity model to these data samples.